In [2]:
!pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: C:\Users\tttoa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: aiohttp, requests, tqdm
Required-by: gpt-index, llama-index, revChatGPT


In [1]:

# from flask import Flask, jsonify, render_template, request, session
import gradio as gr
from openai import OpenAI

# Thay thế 'your-api-key' bằng khóa API của bạn từ OpenAI

client = OpenAI(api_key="")


# app = Flask(__name__)

def format_message(role, content):
    return {"role": role, "content": content}


def get_response(messages):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    content = completion.choices[0].message.content
    return content

user_inputs = [
    "I already know this",
    "I love this",
    "The food at this restaurant was out of this world!",
    "Vô dụng vl",
    "Cảm ơn bạn nhiều nhá",
    "Bạn có thêm nói rõ hơn được không",
    "Cái này mình biết trước rồi"
]

class_inst = f"""
Your task is to classify the comment into one of the following categories:
Positive, Negative

Positive comments are:
- comments provide useful information 

Negative comments are:
- when the comments offend any individual in some types of way
- comments express negative attitude


"""
responses = []

# for user_input in user_inputs:
#     messages = [format_message("system", class_inst),
#                 format_message("user", user_input)]
#     response = get_response(messages)
#     responses.append(response)
#     print(user_input)
#     print(response)

script = """
function createGradioWelcome() {
    // Create a container div
    var container = document.createElement('div');
    container.id = 'gradio-welcome';
    
    // Set styles for the container
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '20px';
    
    // Create a div for the welcome text
    var welcomeText = document.createElement('div');
    welcomeText.innerText = "Comment Classification";
    
    // Append the welcome text to the container
    container.appendChild(welcomeText);
    
    // Find the Gradio container and insert the welcome message at the top
    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);
    
    return 'Welcome message displayed';
}


"""

def classification(text):
    messages = [format_message("system", class_inst),
                format_message("user", text)]
    response = get_response(messages)
    return response


with gr.Blocks(theme='JohnSmith9982/small_and_pretty', js = script) as demo:
    gr.Interface(
    classification,
    inputs=[gr.Textbox(label="Enter your text", lines=3)],
    outputs=[gr.Textbox(label="Classification", lines=3)],
    allow_flagging="never",
    )

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------
IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------


In [25]:
from flask import Flask, jsonify, render_template, request, session
import google.generativeai as genai
from api import Gemini_API_KEY as api
import pandas as pd


genai.configure(api_key="")
model = genai.GenerativeModel('gemini-pro')

app = Flask(__name__)

model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
prompt = """
Your task now is to classify comments as Positive, Negative, or Neutral. 

The data you will work with is a single string read from a CSV file. Each comment in the string is separated by a semi-colon ';'. 

Please classify each comment according to the following criteria:
- Positive: The comment expresses satisfaction, approval, or a positive sentiment.
- Negative: The comment expresses dissatisfaction, disapproval, or a negative sentiment.
- Neutral: The comment is neither positive nor negative, and does not express a strong sentiment.

Here is the example data string: The product is amazing, I am very satisfied!;
 Customer service was terrible, I am very disappointed.; The product is okay, nothing special.;
 I am very pleased with the quality of the product.;
 Delivery was slow and I couldn’t track the order.; Everything is fine, no complaints.

Please provide your classification for each comment in the format:
- [{Comment 1: Setiment 1}, {Comment 2: Setiment 2}, ... ]

Thank you!
"""
chat.send_message(prompt)

chat_history = []

def read_data(path):
    df = pd.read_csv('data/comments.csv')
    all_comments = '; '.join(df['Comment'])
    return all_comments

@app.route('/')
def index():
    return render_template('chat.html', chat_history=chat_history)

@app.route('/chat', methods=['POST'])
def chat_endpoint():
    try:
        user_input = request.json.get('user_input')
        print(user_input)
        if user_input.endswith(".csv"):
            data = read_data(user_input)
            response = chat.send_message(data)
            chat_history.append({"user": user_input, "bot": response.text})

            return jsonify({"response": response.text})
        elif user_input:
            response = chat.send_message(user_input)
            chat_history.append({"user": user_input, "bot": response.text})

            return jsonify({"response": response.text})
        else:
            return jsonify({"error": "No user input provided."}), 400
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [31]:
from openai import OpenAI

# Thay thế 'your-api-key' bằng khóa API của bạn từ OpenAI

client = OpenAI(api_key='')
def chat_with_gpt(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Hoặc mô hình chat khác bạn đang sử dụng
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content

def continuous_chat():
    print("Chào bạn! Hãy bắt đầu trò chuyện với ChatGPT.")
    messages = []
    while True:
        user_input = input("Bạn: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("ChatGPT: Tạm biệt! Hẹn gặp lại.")
            break
        messages.append({"role": "user", "content": user_input})
        response = chat_with_gpt(messages)
        print(f"ChatGPT: {response}")
        messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    continuous_chat()


Chào bạn! Hãy bắt đầu trò chuyện với ChatGPT.
ChatGPT: Chào bạn, có thể tôi giúp gì cho bạn hôm nay?
ChatGPT: Dĩ nhiên, ngày Quốc Khánh của Việt Nam là ngày 2 tháng 9. Đây là ngày kỷ niệm việc độc lập của Việt Nam vào ngày 2 tháng 9 năm 1945.
ChatGPT: Một tuần có 7 ngày: Thứ Hai, Thứ Ba, Thứ Tư, Thứ Năm, Thứ Sáu, Thứ Bảy và Chủ Nhật.
ChatGPT: Chào bạn, có gì tôi có thể giúp bạn không?
ChatGPT: Nếu bạn cần bất kỳ sự trợ giúp nào, đừng ngần ngại để tôi biết! Chúc bạn một ngày tốt lành!
ChatGPT: Nếu bạn có bất kỳ câu hỏi hoặc cần trợ giúp nào, đừng ngần ngại để tôi biết. Chúc bạn một ngày vui vẻ!
